In [1]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 18.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 19.9 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd

df = pd.read_json("/home/andoni/Downloads/output/translated_instruction_0.json")
df['text'] = [str(i) for i in df['text'].values]


In [399]:
df_iter = iter(df.itertuples())

In [539]:
next(df_iter)

Pandas(Index=139, conversation_id=72, text='Zer egingo lukete bazkari kit-ek, nola funtzionatuko lukete horiek? ’')

In [397]:
import re

noise_patterns = [
    r"\bdisplaystyle\b",          # remove 'displaystyle'
    r"[?]{2,}",                   # 3 or more question marks
    r"[-]{2,}",                   # 3 or more dashes
    r"[.]{3,}",                   # 3 or more dots
    r"\s{2,}",                    # 3 or more spaces
    r"(?:\bdisplaystyle\b\s*)+"   # repeated 'displaystyle' sequences
    r"{'translation_text: }"
]

# Compile regexes
noise_regex = re.compile("|".join(noise_patterns), flags=re.IGNORECASE)


def clean_text(text: str) -> str:
    """Remove fillers, noise, and normalize whitespace."""
    if not isinstance(text, str):
        return text
    
    text = text.strip()
    if text.startswith("{'translation_text':") or text.startswith("{\"translation_text\":"):
        text = text.split(":", 1)[1]   # keep everything after first :
        text = text.lstrip(" '\"")     # strip quotes/spaces
        if text.endswith("}"):
            text = text[:-1]
        text = text.rstrip(" '\"")     # strip trailing quotes/spaces

    # Remove noise patterns
    text = noise_regex.sub(" ", text)
    # Normalize spaces
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r":+$", "", text).strip()
    text = re.sub(r"\b(\w+)(?: \1){2,}\b", " ", text)
    return text


In [542]:
for i in range(10):
    df = pd.read_json(f"/home/andoni/Downloads/output/translated_answer_{i}.json")
    df['text'] = [str(j) for j in df['text'].values]
    df["text"] = df["text"].apply(clean_text)
    df.to_json(f"cleaned_answer_{i}.json")


In [398]:
df["text"] = df["text"].apply(clean_text)

In [543]:
df = pd.read_json("/home/andoni/Downloads/output/cleaned_answer_0.json")

In [544]:
df

,conversation_id,text
0,0,Hona hemen osasuntsu egoteko hiru aholku: diet...
1,1,"Hiru kolore nagusiak gorria, urdina eta horia ..."
2,2,"Atomo batek hiru zati nagusi ditu: nukleoa, nu..."
3,3,"Airearen kutsadura murrizteko, garraio publiko..."
4,3,"Uraren kutsadura murrizteko, hondakinak behar ..."
...,...,...
42215,19998,Funtsezko trebetasun bigunak komunikazio eragi...
42216,19998,Ikastetxeek denboraren kudeaketa irakatsi deza...
42217,19998,Ikastetxeek ikasleen aurrerapena ebaluatu deza...
42218,19998,"Ebaluazioei esker, irakasleek denboran zehar i..."


In [16]:
answers = pd.read_json("/home/andoni/Downloads/output/cleaned_answer_0.json")
instructions = pd.read_json("/home/andoni/Downloads/output/cleaned_instruction_0.json")


FileNotFoundError: File /home/andoni/Downloads/output/cleaned_answer_0.json does not exist

In [42]:
for i in range(10):
    answers = pd.read_json(f"cleaned/cleaned_answer_{i}.json")
    instructions = pd.read_json(f"cleaned/cleaned_instruction_{i}.json")
    
    # add turn index (assuming aligned by index)
    instructions["order"] = range(len(instructions))
    answers["order"] = range(len(answers))

    # assign types
    instructions["type"] = "instruction"
    answers["type"] = "answer"

    # map type to numeric priority (instruction first, then answer)
    type_order = {"instruction": 0, "answer": 1}
    instructions["type_order"] = instructions["type"].map(type_order)
    answers["type_order"] = answers["type"].map(type_order)

    # combine and sort properly
    combined = pd.concat([instructions, answers])
    combined = combined.sort_values(by=["conversation_id", "order", "type_order"])

    # group back into conversation list
    conversation_df = combined.groupby("conversation_id") \
        .apply(lambda x: x.text.tolist()) \
        .reset_index(name="conversation")

    conversation_df.to_json(f"conversations/conversation_{i}.json", orient="records")


/tmp/ipykernel_36120/2168970006.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.text.tolist()) \
/tmp/ipykernel_36120/2168970006.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.text.tolist()) \
/tmp/ipykernel_36120/2168970006.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the 

In [554]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 [datasets]/25 [datasets]ce-hub]
Note: you may need to restart the kernel to use updated packages.


In [565]:
df = pd.DataFrame()
conv = pd.read_json(f"conversations/conversation_0.json")


In [566]:
df = pd.concat([df, conv], ignore_index=True, sort=False)

In [567]:
df

,conversation_id,conversation
0,0,[Hiru aholku emango zenizkidake osasuntsu egot...
1,1,[Esango al zenidake zeintzuk diren hiru kolore...
2,2,[Deskriba al dezakezu atomo baten egitura nire...
3,3,[Esango didazue nola murriztu dezakegun kutsad...
4,4,[Esango zenidake erabaki zaila hartu behar zen...
...,...,...
19995,19995,[Zein da bizitzan borrokatzeko helbururik garr...
19996,19996,[Gustatuko al litzaizuke niretzat AI teknologi...
19997,19997,[Iradoki dezakezu film bat benetan indartsua e...
19998,19998,[Nola prestatzen dituzte ikastetxeek ikasleak ...


In [43]:
df = pd.DataFrame()
for i in range(10):
    conv = pd.read_json(f"conversations/conversation_{i}.json")
    df = pd.concat([df, conv], ignore_index=True, sort=False)

In [44]:
df = df.drop('conversation_id', axis=1)


In [45]:
df.to_json('conversations.json')

In [46]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [47]:
dataset

Dataset({
    features: ['conversation'],
    num_rows: 200000
})

In [ ]:
id = "instruct_en_0"

In [ ]:
{
"from": "human",
"speech": "data/multiturn_instruction/en/wav/instruct_en_0/instruct_en_0-1-user.wav",
"text": "Hey, can you give me, like, three tips for staying healthy?",
"unit": null
},

In [584]:
len(df)

200000

In [3]:
import pandas as pd
df = pd.read_json('conversations.json')


In [48]:
df_formated = pd.DataFrame()
# Adibidez daukazun dataframe
# conversation_id | conversation
# 0               | ["instrukzioa", "erantzuna", "beste instrukzio bat", "beste erantzun bat"]

def format_conversation(conversation, idx):
    formatted = []
    for i, text in enumerate(conversation):
        role = "human" if i % 2 == 0 else "gpt"
        formatted.append({
            "from": role,
            "speech": None,
            "text": text,
            "unit": None
        })
    return formatted

# DataFramea sortu edo kargatu
# df = pd.read_json("zure_fitxategia.json")

# ID berria gehitu
df_formated["id"] = [f"instruct_eu_{i}" for i in range(len(df))]

# Conversation berriro formateatu
df_formated["conversation"] = [
    format_conversation(conv, i) for i, conv in enumerate(df["conversation"])
]


In [49]:
df_formated.to_json('conversations.json')

In [50]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_formated)

In [51]:
import huggingface_hub

huggingface_hub.login()

In [31]:
dataset.push_to_hub("Ansu/Instruct_S2S_eu")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/200 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ansu/Instruct_S2S_eu/commit/acd8491eabf89872896180359491f31a29c98ce6', commit_message='Upload dataset', commit_description='', oid='acd8491eabf89872896180359491f31a29c98ce6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ansu/Instruct_S2S_eu', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ansu/Instruct_S2S_eu'), pr_revision=None, pr_num=None)

In [9]:
from datasets import Dataset, load_dataset

data = load_dataset("Ansu/Instruct_S2S_eu")
data = data['train']


In [6]:
data[0]['conversation']

[{'from': 'human',
  'speech': None,
  'text': 'Hiru aholku emango zenizkidake osasuntsu egoteko?',
  'unit': None},
 {'from': 'gpt',
  'speech': None,
  'text': 'Hona hemen osasuntsu egoteko hiru aholku: dieta orekatua jan, ur asko edanez hidratatuta egon eta gauero gutxienez zazpi ordu lo egin.',
  'unit': None}]

In [ ]:
data['train']['id']

200000

In [30]:
def add_speech_paths(example):
    new_conversation = []
    i = 0
    for turn in example["conversation"]:
        new_turn = turn.copy()
        if turn["from"] == "human":
            new_turn["speech"] = f"{example['id']}_user_{i}.wav"
        elif turn["from"] == "gpt":
            new_turn["speech"] = f"{example['id']}_assistant_{i}.wav"
            i += 1
        new_conversation.append(new_turn)
    example["conversation"] = new_conversation
    return example

# if your dataset is a Dataset
dataset = data.map(add_speech_paths)



Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

In [29]:
dataset[9]['conversation']

[{'from': 'human',
  'speech': 'instruct_eu_9_user_0.wav',
  'text': 'Ebaluatu dezakezu esaldi hau ortografia eta gramatika akatsak dio bere bazkaria finnished zuen eta reurant utzi zuen?',
  'unit': None},
 {'from': 'gpt',
  'speech': 'instruct_eu_9_assistant_0.wav',
  'text': 'Esaldiak bi akats ditu, esaldi zuzena irakurri beharko litzateke: Bazkaria amaitu eta jatetxetik irten zen.',
  'unit': None},
 {'from': 'human',
  'speech': 'instruct_eu_9_user_1.wav',
  'text': 'Egiaztatu esaldi hau akatsak aurkitzeko: Libariara joan dira oraindik ere.',
  'unit': None},
 {'from': 'gpt',
  'speech': 'instruct_eu_9_assistant_1.wav',
  'text': 'Esaldiak bi errore ditu. Esaldi zuzenak hauxe irakurri beharko luke: Oraindik ez dira liburutegira joan.',
  'unit': None},
 {'from': 'human',
  'speech': 'instruct_eu_9_user_2.wav',
  'text': 'Egiaztatu esaldi hau akatsak egiteko: Teknikariak etxeko lanak egin dizkigu gaur gauean.',
  'unit': None},
 {'from': 'gpt',
  'speech': 'instruct_eu_9_assistant_